# Necessary imports

This was my top scoring model, inspired by https://github.com/Project-MONAI/tutorials/blob/main/3d_segmentation/swin_unetr_btcv_segmentation_3d.ipynb, then augmented using literature and asking Gemini to implement some changes to the configuration and transforms.

In [ ]:
import torch
import torch.optim as optim
from monai.losses import DiceCELoss # Use combined loss
from monai.metrics import DiceMetric
from monai.networks.nets import UNet
from monai.data import Dataset, DataLoader, decollate_batch
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, Spacingd, CropForegroundd,
    Resized, DivisiblePadd, ToTensord, RandFlipd, RandAffined, RandScaleIntensityd,
    RandShiftIntensityd, ScaleIntensityRangePercentilesd, EnsureType, AsDiscrete,
    RandGaussianNoised, RandAdjustContrastd
)
from monai.utils import set_determinism, first
from monai.networks.nets import SwinUNETR

from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import AdamW 

import os
from glob import glob
# import time
import matplotlib.pyplot as plt
import numpy as np

from monai.inferers import sliding_window_inference # Keep this

Configuration and file paths

In [ ]:
SEED = 42
set_determinism(seed=SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

data_dir = '/datasets/tdt4265/mic/open/HNTS-MRG'
CHECKPOINT_DIR = "./checkpoints_swin_v2"
BEST_MODEL_PATH = f"{CHECKPOINT_DIR}/best_metric_model_swin_v2.pth"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Data Loading ---
train_images = sorted([f for f in glob(os.path.join(data_dir, 'train', '*', 'preRT', '*.nii.gz')) if '_mask' not in f])
train_labels = sorted([f for f in glob(os.path.join(data_dir, 'train', '*', 'preRT', '*.nii.gz')) if '_T2' not in f])
val_images = sorted([f for f in glob(os.path.join(data_dir, 'test', '*', 'preRT', '*.nii.gz')) if '_mask' not in f])
val_labels = sorted([f for f in glob(os.path.join(data_dir, 'test', '*', 'preRT', '*.nii.gz')) if '_T2' not in f])

train_files = [{'image': image_name, 'label': label_name} for image_name, label_name in zip(train_images, train_labels)]
val_files = [{'image': image_name, 'label': label_name} for image_name, label_name in zip(val_images, val_labels)]

Transform data

In [ ]:
# Standardize the dimensions
TARGET_SPACING = (1.5, 1.5, 2.0)
TARGET_SIZE = (160, 160, 96)
PAD_K = 16 

train_transforms = Compose([
    LoadImaged(keys=['image', 'label']),
    EnsureChannelFirstd(keys=['image', 'label']),
    Spacingd(keys=['image', 'label'], pixdim=TARGET_SPACING, mode=('bilinear', 'nearest')),
    # *** RE-ENABLED CropForegroundd *** (Crucial)
    CropForegroundd(keys=['image', 'label'], source_key='image', margin=10),
    ScaleIntensityRangePercentilesd(keys='image', lower=0.5, upper=99.5,
                               b_min=0.0, b_max=1.0, clip=True),

    # Augmentation:
    RandFlipd(keys=['image','label'], spatial_axis=0, prob=0.5), 
    RandFlipd(keys=['image','label'], spatial_axis=1, prob=0.5),
    RandFlipd(keys=['image','label'], spatial_axis=2, prob=0.5), 
    RandAffined(
        keys=['image','label'],
        rotate_range=(0.1, 0.1, 0.1), 
        scale_range=(0.2, 0.2, 0.2), 
        mode=('bilinear','nearest'),
        prob=0.3 
    ),
    RandGaussianNoised(keys='image', prob=0.2, mean=0.0, std=0.05),
    RandAdjustContrastd(keys='image', prob=0.2, gamma=(0.7, 1.3)),
    RandScaleIntensityd(keys='image', factors=0.2, prob=0.2), 
    RandShiftIntensityd(keys='image', offsets=0.15, prob=0.3),

    Resized(keys=['image','label'], spatial_size=TARGET_SIZE, mode=('bilinear', 'nearest')),
    DivisiblePadd(keys=['image','label'], k=PAD_K, mode='constant', constant_values=0),
    ToTensord(keys=['image','label'])
])

val_transforms = Compose(
    [
        LoadImaged(keys=['image', 'label']),
        EnsureChannelFirstd(keys=['image', 'label']),
        Spacingd(keys=['image', 'label'], pixdim=TARGET_SPACING, mode=('bilinear', 'nearest')),
        CropForegroundd(keys=['image', 'label'], source_key='image', margin=10),
        ScaleIntensityRangePercentilesd(keys='image', lower=0.5, upper=99.5,
                               b_min=0.0, b_max=1.0, clip=True),
        Resized(keys=['image', 'label'], spatial_size=TARGET_SIZE, mode=('bilinear', 'nearest')),
        DivisiblePadd(keys=['image', 'label'], k=PAD_K, mode='constant', constant_values=0),
        ToTensord(keys=['image', 'label'])
    ]
)

train_dataset = Dataset(data=train_files, transform=train_transforms)
train_loader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=True, num_workers=4)

val_dataset = Dataset(data=val_files, transform=val_transforms)
val_loader = DataLoader(dataset=val_dataset, batch_size=1, num_workers=4)

Model definition

In [ ]:
model = SwinUNETR(
    img_size=TARGET_SIZE,
    in_channels=1,
    out_channels=3,       # Background, Class1, Class2
    feature_size=48,      # Standard feature size
    dropout_path_rate=0.2, # Keep this for now, can tune later
    use_checkpoint=True,   # Use gradient checkpointing to save memory if needed
).to(device)

ckpt = torch.load("model_swinvit.pt", map_location=device)
# Check if the checkpoint has a 'state_dict' key or is the state_dict directly
if 'state_dict' in ckpt:
    state_dict = ckpt['state_dict']
else:
    state_dict = ckpt

print("Loaded pretrained Swin UNETR weights!")

Training setup

In [ ]:
max_epochs = 300
val_interval = 5
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
lr_values = []

post_pred = Compose([EnsureType(), AsDiscrete(argmax=True, to_onehot=3)])
post_label = Compose([EnsureType(), AsDiscrete(to_onehot=3)])
loss_function = DiceCELoss(to_onehot_y=True, softmax=True, include_background=True)

dice_metric = DiceMetric(include_background=False, reduction="mean_batch", get_not_nans=True) # Focus metric on foreground
dice_metric_bg = DiceMetric(include_background=True, reduction="mean_batch", get_not_nans=True) # Separate metric for full reporting

# Optimizer
initial_lr = 1e-4
optimizer = AdamW(model.parameters(), lr=initial_lr, weight_decay=1e-5) # Start with same WD, tune if needed

# Warmup as recommended by UDL
warmup_epochs = 10

# Scheduler
scheduler = CosineAnnealingLR(optimizer, T_max=max_epochs - warmup_epochs, eta_min=1e-6) 
# See later that MONAI has builtin WarmupCosineSchedule, maybe could've used that instead

Validation function

In [ ]:
def run_validation(model, val_loader, post_pred, post_label, dice_metric, dice_metric_bg, device, epoch):
    model.eval()
    with torch.no_grad():
        for val_data in val_loader:
            images = val_data["image"].to(device)
            labels = val_data["label"].to(device)

            # *** Use overlap in sliding window inference ***
            outputs = sliding_window_inference(
                images, roi_size=TARGET_SIZE, sw_batch_size=2, predictor=model, overlap=0.5
            )

            # post‐process into lists of tensors
            y_pred = [post_pred(x) for x in decollate_batch(outputs)]
            y_true = [post_label(x) for x in decollate_batch(labels)]

            # Calculate metrics
            dice_metric(y_pred=y_pred, y=y_true)
            dice_metric_bg(y_pred=y_pred, y=y_true)

    # Aggregate metrics
    metric_foreground, _ = dice_metric.aggregate()
    metric_all, _ = dice_metric_bg.aggregate()
    dice_metric.reset()
    dice_metric_bg.reset()

    # Extract per-class Dice (assuming metric_all gives [bg, c1, c2])
    # Handle potential case where a class is not present in any validation sample
    dice_bg = metric_all[0].item() if len(metric_all)>0 else 0
    dice_cls1 = metric_all[1].item() if len(metric_all)>1 else 0
    dice_cls2 = metric_all[2].item() if len(metric_all)>2 else 0

    # Average foreground Dice
    avg_dice_foregrounds = metric_foreground.mean().item()

    # Store detailed metrics
    metric_values.append((
        epoch + 1,
        dice_bg,
        dice_cls1,
        dice_cls2,
        avg_dice_foregrounds # This is the primary metric for comparison now
    ))

    print(f"Dice(background): {dice_bg:.4f}, Dice(class1): {dice_cls1:.4f}, "
          f"Dice(class2): {dice_cls2:.4f}, Avg(foregrounds): {avg_dice_foregrounds:.4f}")

    return avg_dice_foregrounds

Training loop

In [ ]:
early_stopping_patience = 25 
early_stopping_counter = 0

print(f"Starting training for {max_epochs} epochs...")

for epoch in range(max_epochs):
    print(f"\n--- Epoch {epoch+1}/{max_epochs} ---")
    model.train()
    running_loss = 0.0
    current_lr = optimizer.param_groups[0]['lr']

    # Manual Warmup Phase
    if epoch < warmup_epochs:
        # Simple linear warmup
        warmup_factor = (epoch + 1) / warmup_epochs
        current_lr = initial_lr * warmup_factor
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_lr
        print(f"  Warmup: LR set to {current_lr:.6f}")
    elif epoch == warmup_epochs:
         print(f"  Warmup finished. Cosine annealing starts.")

    for step, batch_data in enumerate(train_loader, 1):
        images = batch_data["image"].to(device)
        labels = batch_data["label"].to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        # Optional: Gradient clipping (can help stabilize training)
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item()
        if step % 5 == 0: # Print less frequently
             print(f"  step {step:>3}/{len(train_loader):<3}  loss: {loss.item():.4f}")

    # Step the scheduler *after* the optimizer step, only *after* warmup phase
    if epoch >= warmup_epochs:
         scheduler.step()

    avg_train_loss = running_loss / len(train_loader)
    print(f"  Average Train Loss: {avg_train_loss:.4f}")
    epoch_loss_values.append((epoch+1, avg_train_loss))
    lr_values.append((epoch+1, current_lr)) # Log the actual LR used

    # --- Validation ---
    if (epoch + 1) % val_interval == 0:
        val_metric = run_validation(model, val_loader, post_pred, post_label, dice_metric, dice_metric_bg, device, epoch)
        print(f"  Validation Avg Foreground Dice: {val_metric:.4f}   (best: {best_metric:.4f} at epoch {best_metric_epoch})")

        # Early stopping logic (based on foreground Dice)
        if val_metric > best_metric:
            best_metric = val_metric
            best_metric_epoch = epoch + 1
            early_stopping_counter = 0 # Reset counter

            torch.save(
                {
                    "epoch": epoch + 1,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "scheduler_state_dict": scheduler.state_dict() if scheduler else None, # Save scheduler state
                    "best_metric": best_metric,
                },
                BEST_MODEL_PATH,
            )
            print(f" best model saved at epoch {best_metric_epoch}!")

        else:
            early_stopping_counter += 1
            print(f"No improvement. Counter: {early_stopping_counter}/{early_stopping_patience}")
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered after {early_stopping_patience * val_interval} epochs ({early_stopping_counter} checks) without improvement.")
                break # Stop training

print(f"\nTraining finished. Best validation foreground Dice: {best_metric:.4f} at epoch {best_metric_epoch}")

Plot training and validation loss

In [ ]:
import matplotlib.pyplot as plt

# Unpack the data from the metric collections.
# For the dice metrics (validation), each element in metric_values is:
# (epoch, dice_bg, dice_cls1, dice_cls2, avg_dice_foregrounds)
epochs_dice, dice_bg, dice_cls1, dice_cls2, avg_dice_fg = zip(*metric_values)

# For the training loss (average in-sample error), each element in epoch_loss_values is:
# (epoch, avg_train_loss)
epochs_loss, train_loss = zip(*epoch_loss_values)

# Create a figure with two y-axes.
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot Dice scores on the primary y-axis.
ax1.plot(epochs_dice, dice_bg, 'b-o', label='Dice (Background)')
ax1.plot(epochs_dice, dice_cls1, color='orange', marker='o', linestyle='-', label='Dice (Class 1)')
ax1.plot(epochs_dice, dice_cls2, 'g-o', label='Dice (Class 2)')
ax1.plot(epochs_dice, avg_dice_fg, 'k--o', label='Avg Dice (Foregrounds)')
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Dice Score")
ax1.set_ylim(0, 1)
ax1.tick_params(axis='y')

# Create a second y-axis for the training loss.
ax2 = ax1.twinx()
ax2.plot(epochs_loss, train_loss, 'r-s', linestyle='--', label='Train Loss (In-sample Error)')
ax2.set_ylabel("Train Loss")
ax2.tick_params(axis='y', labelcolor='red')
ax2.set_ylim(0, max(train_loss) * 1.1)  # adjust as needed

# Combine legends from both axes.
lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='best')

plt.title("Validation Dice Scores and Train Loss per Epoch")
plt.grid(True)
plt.tight_layout()